In [15]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [16]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'df41ec0ab766a80fabc93cbf3e2f0200', 'Date': 'Wed, 21 Aug 2024 20:33:30 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [17]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Carter',
  'subject': 'Introduction to Computer Science',
  'stars': 5,
  'review': 'Dr. Carter is amazing at explaining complex concepts in a simple way. Her classes are engaging and her office hours are very helpful.'},
 {'professor': 'Prof. David Nguyen',
  'subject': 'Software Engineering',
  'stars': 4,
  'review': 'Prof. Nguyen is a great teacher who encourages hands-on learning. His projects are fun but can be a bit overwhelming at times.'},
 {'professor': 'Dr. Sarah Mitchell',
  'subject': 'Algorithms and Data Structures',
  'stars': 3,
  'review': 'Dr. Mitchell is very knowledgeable but her lectures can be a bit dry. Be prepared to put in extra study hours.'},
 {'professor': 'Dr. James Patel',
  'subject': 'Database Systems',
  'stars': 4,
  'review': "Dr. Patel is thorough and detailed in his lectures. He can be strict, but if you follow along and ask questions, you'll do well."},
 {'professor': 'Prof. Maria Rossi',
  'subject': 'Operating Systems',


In [20]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model = "text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [21]:
processed_data[0]

{'values': [-0.031852163,
  -0.0034369077,
  -0.0073661203,
  0.06212895,
  0.032787543,
  0.024209121,
  0.008849191,
  0.054744367,
  -0.018522993,
  0.0028661408,
  0.024652196,
  0.0014707625,
  0.02187067,
  -0.012529174,
  -0.00069807377,
  0.010683029,
  -0.0025846036,
  -0.013673784,
  0.04452903,
  0.02525527,
  -0.007156891,
  -0.00090614974,
  0.055433597,
  0.030301401,
  -0.036479834,
  -0.031729087,
  0.034756765,
  0.035593685,
  0.0133537855,
  0.015003009,
  0.09097805,
  -0.020762984,
  -0.020024525,
  -0.03798137,
  -0.06311356,
  0.06995661,
  -0.008215347,
  0.020147601,
  0.035002917,
  -0.0003271139,
  0.021538364,
  0.048762858,
  -0.0027538338,
  0.0010799951,
  0.035593685,
  -0.010215338,
  -0.041304428,
  0.0048953625,
  0.031532165,
  0.03864598,
  -0.076701194,
  -0.010873797,
  0.023310663,
  -0.007286121,
  -0.060553573,
  0.049870543,
  0.0037169063,
  0.005043054,
  0.01348917,
  -0.036750603,
  0.052676685,
  -0.005150746,
  -0.018166073,
  -0.0092245

In [22]:
index = pc.Index('rag')
index.upsert(
    vectors = processed_data,
    namespace = "ns1"
)

{'upserted_count': 20}

In [23]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}